In [1]:
# AUTO ARIMA
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statsmodels.tsa.arima_model import ARIMA
from arch import arch_model
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')
sns.set()

In [5]:
# Importing the data
raw_data = yf.download(tickers = '000063.SZ 000661.SZ 600584.SS 600588.SS 600690.SS',
                       start='2005-01-02', end='2022-01-01', period = 'max', interval = '1d', group_by = 'ticker', auto_adjust = True, prepost = True, threads = True, proxy = None)

[*********************100%***********************]  5 of 5 completed


In [23]:
df_comp = raw_data.copy()

In [24]:
tickers = ['000063.SZ', '000661.SZ', '600584.SS', '600588.SS', '600690.SS']
for i in tickers:
    df_comp[f"{i}"] = df_comp[f"{i}"].Close[:]
df_comp = df_comp.iloc[1:]
#for i in tickers:
#    del df_comp[f"{i}"]
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')

In [25]:
df_comp

600690.SS                                              600588.SS  \
                 Open       High        Low      Close     Volume       Open   
Date                                                                           
2005-01-04   0.798385   0.798385   0.798385   0.798385   0.798385   0.541001   
2005-01-05   0.804623   0.804623   0.804623   0.804623   0.804623   0.548957   
2005-01-06   0.804623   0.804623   0.804623   0.804623   0.804623   0.541001   
2005-01-07   0.813979   0.813979   0.813979   0.813979   0.813979   0.592449   
2005-01-10   0.809301   0.809301   0.809301   0.809301   0.809301   0.584758   
...               ...        ...        ...        ...        ...        ...   
2021-12-27  28.363363  28.363363  28.363363  28.363363  28.363363  36.112236   
2021-12-28  29.776136  29.776136  29.776136  29.776136  29.776136  35.803921   
2021-12-29  28.696936  28.696936  28.696936  28.696936  28.696936  35.237026   
2021-12-30  29.118805  29.118805  29.118805  29.118805  29.118805  35.774086   
2021-12-31  29.324833  29.324833  29.324833  29.324833  29.324833  35.684578   

                                                        ...   000661.SZ  \
                 High        Low      Close     Volume  ...        Open   
Date                                                    ...               
2005-01-04   0.541001   0.541001   0.541001   0.541001  ...    1.750358   
2005-01-05   0.548957   0.548957   0.548957   0.548957  ...    1.808582   
2005-01-06   0.541001   0.541001   0.541001   0.541001  ...    1.750358   
2005-01-07   0.592449   0.592449   0.592449   0.592449  ...    1.837694   
2005-01-10   0.584758   0.584758   0.584758   0.584758  ...    1.837694   
...               ...        ...        ...        ...  ...         ...   
2021-12-27  36.112236  36.112236  36.112236  36.112236  ...  276.408051   
2021-12-28  35.803921  35.803921  35.803921  35.803921  ...  280.568878   
2021-12-29  35.237026  35.237026  35.237026  35.237026  ...  273.919495   
2021-12-30  35.774086  35.774086  35.774086  35.774086  ...  270.117004   
2021-12-31  35.684578  35.684578  35.684578  35.684578  ...  270.156799   

                                                            000063.SZ  \
                  High         Low       Close      Volume       Open   
Date                                                                    
2005-01-04    1.750358    1.750358    1.750358    1.750358   5.072096   
2005-01-05    1.808582    1.808582    1.808582    1.808582   5.013493   
2005-01-06    1.750358    1.750358    1.750358    1.750358   4.896284   
2005-01-07    1.837694    1.837694    1.837694    1.837694   4.818775   
2005-01-10    1.837694    1.837694    1.837694    1.837694   4.780965   
...                ...         ...         ...         ...        ...   
2021-12-27  276.408051  276.408051  276.408051  276.408051  32.700855   
2021-12-28  280.568878  280.568878  280.568878  280.568878  32.562672   
2021-12-29  273.919495  273.919495  273.919495  273.919495  32.246811   
2021-12-30  270.117004  270.117004  270.117004  270.117004  32.552799   
2021-12-31  270.156799  270.156799  270.156799  270.156799  33.066063   

                                                        
                 High        Low      Close     Volume  
Date                                                    
2005-01-04   5.072096   5.072096   5.072096   5.072096  
2005-01-05   5.013493   5.013493   5.013493   5.013493  
2005-01-06   4.896284   4.896284   4.896284   4.896284  
2005-01-07   4.818775   4.818775   4.818775   4.818775  
2005-01-10   4.780965   4.780965   4.780965   4.780965  
...               ...        ...        ...        ...  
2021-12-27  32.700855  32.700855  32.700855  32.700855  
2021-12-28  32.562672  32.562672  32.562672  32.562672  
2021-12-29  32.246811  32.246811  32.246811  32.246811  
2021-12-30  32.552799  32.552799  32.552799  32.552799  
2021-12-31  33.066063  33.066063  33.066063  33.066063  

[4434 rows x 25 co